# Лабораторная работа №4 - Алгоритм Карплуса-Стронга

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as display

Класс <b>KSConfig</b> описывает конфигурацию алгоритма - множители при $y_{n-N}$ и $y_{n-N-1}$, начальную последовательность $y_0, y_1, ... y_{N-1}$, а также частоту дискретизации (в данной работе она фиксирована - 22050 Гц).

In [2]:
class KSConfig:
    def __init__(self,
                 fd:int=22050,
                 c:tuple=(0.5, 0.5),
                 init_seq:str='normal'):
        self.config = dict()
        self.config['fd'] = fd
        self.config['c'] = c
        self.config['init_seq'] = init_seq
        
    def get_config(self):
        return self.config

def triangle(x):
    if -np.pi <= x < -np.pi/2:
        return -2/np.pi * x - 2
    elif -np.pi/2 <= x < np.pi/2:
        return 2/np.pi * x
    elif np.pi/2 <= x <= np.pi:
        return -2/np.pi * x + 2
    return 0

def init_seq_gen(count, seq_type='normal'):
    if seq_type == 'normal':
        return np.random.normal(0, 1, count)
    elif seq_type == 'uniform':
        return np.random.uniform(-0.5, 0.5, count)
    elif seq_type == 'sine':
        return np.sin(np.linspace(0, 2*np.pi, count))
    elif seq_type == 'triangle':
        return np.vectorize(triangle)(np.linspace(-np.pi, np.pi, count))
    return None

def karplus_strong(config, f0, T):
    fd, c, init_seq = config.values()
    K = int(T*fd)
    y = np.zeros(K)
    P = int(np.round(fd/f0))
    
    y[:P+1] = init_seq_gen(P+1, init_seq)
    for k in range(P+1, K):
        y[k] = c[0]*y[k-P] + c[1]*y[k-P-1]
    return y

def melody(config:KSConfig, notes:list=[0], delay=0.04, base_freq=130.82, dur=3):
    conf_dict = config.get_config()
    _delay = int(delay*conf_dict['fd'])
    K = int(dur*conf_dict['fd'])
    x = np.zeros((K + (len(notes)-1)*_delay, len(notes)))
    padding = np.zeros(_delay)
    
    for i, k in enumerate(notes):
        if k is None:
            x[i*_delay:len(x[:,i])-(len(notes)-1-i)*_delay,i] = np.zeros(K)
        else:
            f0 = base_freq * np.power(2, k/12)
            x[i*_delay:len(x[:,i])-(len(notes)-1-i)*_delay,i] = karplus_strong(conf_dict, f0, dur)
    return x

Доступные формы волны:
- нормальное распределение (normal)
- равномерное распределение (uniform)
- синусоидальная (sine)
- треугольная (triangle)

Коэффициенты при $y_{n-N}$ и $y_{n-N-1}$ определяются пользователем при создании конфига.

In [3]:
fd = 22050

ro = 0.988
alpha = 0.8
ro_alpha = 0.995

cfg_default = KSConfig()

# Разные формы волны
cfg_wf_uniform = KSConfig(init_seq="uniform")
cfg_wf_sine = KSConfig(init_seq="sine")
cfg_wf_triangle = KSConfig(init_seq="triangle")

# Варьирование длительности затухания
cfg_fade_faster = KSConfig(c=(ro/2, ro/2))
cfg_fade_slower = KSConfig(c=(ro_alpha*(1-alpha), ro_alpha*alpha))

Ниже представлено четыре варианта одного и того же аккорда с разной начальной последовательностью.

In [4]:
chord = melody(cfg_default, [3, 7, 10])
result = np.sum(chord, axis=1)
print("Нормальное распределение:")
display.display(display.Audio(result, rate=fd))

chord = melody(cfg_wf_uniform, [3, 7, 10])
result = np.sum(chord, axis=1)
print("Равномерное распределение:")
display.display(display.Audio(result, rate=fd))

chord = melody(cfg_wf_sine, [3, 7, 10])
result = np.sum(chord, axis=1)
print("Синусоида:")
display.display(display.Audio(result, rate=fd))

chord = melody(cfg_wf_triangle, [3, 7, 10])
result = np.sum(chord, axis=1)
print("Волна треугольной формы:")
display.display(display.Audio(result, rate=fd))

Нормальное распределение:


Равномерное распределение:


Синусоида:


Волна треугольной формы:


Синусоидальная и треугольная последовательности приглушают сигнал, а нормальное и равномерное распределение позволяют сделать его отчетливым.

Также равномерная и треугольная последовательности делают сигнал более выраженным, что позволяет предположить, что простые последовательности лучше подходят для имитации звука струны акустической гитары.

Ниже представлены модификации алгоритма, осуществленные посредством изменения стандартных коэффициентов в рекуррентной формуле. В первом случае длительность затухания сигнала с низкой базовой частотой уменьшается (изначально он слишком долго затухает), а во втором - длительность, наобоброт, увеличивается, поскольку без модификаций в таком случае сигнал затухает слишком быстро.

In [5]:
chord = melody(cfg_fade_faster, [3, 7, 10], base_freq=62.1)
result = np.sum(chord, axis=1)
display.display(display.Audio(result, rate=fd))

chord = melody(cfg_fade_slower, [3, 7, 10], base_freq=262)
result = np.sum(chord, axis=1)
display.display(display.Audio(result, rate=fd))

Несмотря на наличие модификаций, коэффициенты $\rho$ в любом случае близки к 1 - иначе затухание происходит слишком быстро даже для низких частот. $\alpha$ также заметно больше 0.5. В ходе экспериментов с алгоритмом наблюдалось, что он чувствителен к изменению коэффициентов, поэтому надежнее не выходить из диапазона частот, на котором алгоритм дает приемлемый результат.

Функция <b>melody</b> предоставляет возможность гибкой настройки воспроизведения нот:

In [6]:
sample_melody = [8, 12, 15,
                 None, None, 8, 12, 15,
                 0, 8, 7,
                 None, None, 3, 7, 8]
sample = melody(cfg_default, sample_melody, delay=0.3)
result = np.sum(sample, axis=1)
display.display(display.Audio(result, rate=fd))